In [2]:
from google.colab import drive
drive.mount('/content/drive')
% cd '/content/drive/My Drive/785_proj/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/785_proj


In [3]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
!pip install import-ipynb
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import import_ipynb


num_workers = 0
batch_size = 32
transform = transforms.Compose(
    [transforms.Resize((64,64)),
     transforms.ToTensor(),
     ])

train_data = datasets.ImageFolder(root = 'crop_part1', transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2975 sha256=a5c6b4c368e897bd26da76eaba957648d1db2102e5b09c3b1e0f3e0318cc55a0
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [0]:
from GeneratorUnet import * 
import DiscriminatorGender
from torchvision import models
import torch

G = UNet(3,3).to(device)
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.classifier = nn.Sequential(
    nn.Linear(1280, 2),
    nn.Softmax(),
)
D_F = mobilenet
D_G = torch.load('GD.pt')
# set gender discriminator to be fixed
for param in D_G.parameters():
    param.requires_grad = False

In [0]:
"""
define loss
"""
def l2_loss_G(out, input):
    criterion = nn.MSEloss()
    loss = criterion(out, input)
    return loss

def gender_D_loss(out, label):
    batch_size = out.size(0)
    if label == 0:
        labels = torch.zeros(batch_size)
    if label == 1:
        labels = torch.ones(batch_size)
    criterion = nn.BCEloss()
    return criterion(out, labels)

def true_D_loss(out):
    batch_size = out.size(0)
    labels = torch.ones(batch_size)
    criterion = nn.BCELoss()
    return criterion(out, labels)

def fake_D_loss(out):
    batch_size = out.size(0)
    labels = torch.ones(batch_size)
    criterion = nn.BCELoss()
    return criterion(out, labels)

In [0]:
import torch.optim as optim

lr = 0.002
d_optimizer = optim.Adam(D_F.parameters(), lr)
g_optimizer = optim.Adam(G.parameters(), lr)

In [0]:
# training
# training hyperparams
num_epochs = 100

# keep track of loss and generated, "fake" samples
samples = []
losses = []

print_every = 10

# train the network
D_F.train()
D_G.train()
G.train()
for epoch in range(num_epochs):
    
    for batch_i, (real_images, genders) in enumerate(train_loader):
                
        batch_size = real_images.size(0)

        # ============================================
        #            TRAIN THE DISCRIMINATOR
        # ============================================
        
        d_optimizer.zero_grad()
        
        # 1. Train with real images

        # Compute the discriminator losses on real images 
        D_real = D(real_images)
        real_loss = true_D_loss(D_real)
        
        # 2. Train with fake images
        
        # Generate fake female images
        fake_images_f = G(real_images, 0)
        fake_images_m = G(real_images, 1)
        # Compute the discriminator losses on fake images        
        D_fake_f = D(fake_images_f)
        d_fake_f_loss = fake_loss(D_fake)
        D_fake_m = D(fake_images_m)
        d_fake_m_loss = fake_loss(D_fake_m)

        # add up loss and perform backprop
        d_loss = d_real_loss + d_fake_f_loss + d_fake_m_loss
        d_loss.backward()
        d_optimizer.step()

        # =========================================
        #            TRAIN THE GENERATOR
        # =========================================
        g_optimizer.zero_grad()
        
        # 1. Train with l2loss
        # Generate fake female images
        fake_images_f = G(real_images, 0)
        fake_images_m = G(real_images, 1)
        l2loss_f = l2_loss_G(fake_images_f, real_images)
        l2loss_m = l2_loss_G(fake_images_m, real_images)

        # using flipped labels!

        # 2. Train with gender discriminator
        D_G_fake_f = D_G(fake_images_f)
        D_G_fake_m = D_G(fake_images_m)

        gender_loss_f = gender_d_loss(D_G_fake_f, 0)
        gender_loss_m = gender_d_loss(D_G_fake_m, 1)
        
        # 3. Train with face discriminator
        D_F_fake_f = D_F(fake_images_f)
        D_F_fake_m = D_F(fake_images_m)

        face_loss_f =  true_D_loss(D_F_fake_f)
        face_loss_m =  true_D_loss(D_F_fake_m)

        # sum all the loss
        g_loss = l2loss_f + l2loss_m + gender_loss_f + gender_loss_m + face_loss_f + face_loss_m
        # perform backprop
        g_loss.backward()
        g_optimizer.step()

        # Print some loss stats
        if batch_i % print_every == 0:
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, d_loss.item(), g_loss.item()))
            
        ## AFTER EACH EPOCH##
        # append discriminator loss and generator loss
        losses.append((d_loss.item(), g_loss.item()))
        
        # generate and save sample, fake images
        G.eval() # eval mode for generating samples
        samples_z = G(fixed_z)
        samples.append(samples_z)
        G.train() # back to train mode


In [8]:
epochs = 10
train(Gmodel, train_loader)

NameError: ignored